In [1]:
# © 2025 Nicolas Bosc <nbosc@ebi.ac.uk>

# Extract PK parameters from ChEMBL assay descriptions
<b>Prerequisite:</b> assuming pandas is already installed, `pip install chembl-downloader`

<b>Objective:</b> Using regex to extract PK/PD parameters (e.g. dose, time, route of admistration etc.) from ChEMBL assay descriptions.

<b>Description:</b>
- The first part queries the ChEMBL database for the relevant data and extract some PK/PD parameters from the assay descriptions
- The second part applies some sanity checks, and standardisation rules to extracted parameters prior to loading the data in ChEMBL.

<b>NB:</b> This notebook does not describe the loading part.

In [2]:
import pandas as pd, numpy as np
import re
import pk_parameters_regex as pk_regex

import chembl_downloader

import warnings
warnings.filterwarnings('ignore')

#### notebook settings
# pd.options.display.max_rows= 100
# pd.options.display.max_columns = 50
# pd.options.display.max_colwidth = 100

/opt/surechembl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Direct ChEMBL database access 

In [3]:
path = chembl_downloader.download_extract_sqlite(version='33')

## Functions 

# PART 1

## Regex 

In [4]:
dose_unit = pk_regex.dose_unit
time_of_measurement = pk_regex.time_of_measurement
route_of_administration = pk_regex.route_of_administration
tissue = pk_regex.tissue

##  Legacy data extracted from assay description

#### The PK parameters (time, route of administration, dose etc.) might be written in the assay description and they can be extracted using regular expressions.

#### Legacy data definition
- Data from CHEMBL 33
- standard_type = Cmax or AUC


In [5]:
sql='''
SELECT a.activity_id, a.standard_type,
       b.assay_id,b.description, b.assay_type
FROM
    activities a
    JOIN assays b ON a.assay_id = b.assay_id
WHERE
    lower(a.standard_type) IN ('cmax', 'auc')
'''
df_pk = chembl_downloader.query(sql, version = '33')
df_pk.shape

(61779, 5)

In [6]:
df_cmax = df_pk[df_pk.standard_type=='Cmax']
df_cmax.shape

(24692, 5)

In [7]:
df_auc= df_pk[df_pk.standard_type=='AUC']
df_auc.shape

(37087, 5)

In [8]:
df_cmax.activity_id.nunique()

24692

In [9]:
df_auc.activity_id.nunique()

37087

#### Use the regex to get the PK parameters

In [10]:
df_cmax = df_cmax.assign(
    dose_unit=df_cmax.apply(lambda x: pk_regex.find_dose_unit(dose_unit, x['description']), axis=1),
    route=df_cmax.apply(lambda x: pk_regex.find_route(route_of_administration, x['description']), axis=1),
    tissue=df_cmax.apply(lambda x: pk_regex.find_tissue(tissue, x['description']),axis=1),
    )

In [11]:
df_auc = df_auc.assign(
    dose_unit=df_auc.apply(lambda x: pk_regex.find_dose_unit(dose_unit, x['description']), axis=1),
    route=df_auc.apply(lambda x: pk_regex.find_route(route_of_administration, x['description']), axis=1),
    time=df_auc.apply(lambda x: pk_regex.find_time(time_of_measurement, x['description']), axis=1),
    tissue=df_auc.apply(lambda x: pk_regex.find_tissue(tissue, x['description']),axis=1),
    )

# Part 2

In [12]:
df_auc = pk_regex.remove_low_quality_pk_data(df_auc)

DATA FILTERING - START

Before curation, there are 37087 AUC data points.
1150 AUC data points have an ambiguous route of administration and were excluded.
2803 AUC data points have no dose information but were kept.
228 AUC data points have a dose range and they have been excluded.
14908 AUC data points have no time and they have been excluded
After curation, there are now 20800 AUC data points

DATA FILTERING - END


In [13]:
df_auc = pk_regex.remove_low_quality_pk_data(df_auc)

DATA FILTERING - START

Before curation, there are 20800 AUC data points.
0 AUC data points have an ambiguous route of administration and were excluded.
771 AUC data points have no dose information but were kept.
0 AUC data points have a dose range and they have been excluded.
0 AUC data points have no time and they have been excluded
After curation, there are now 20800 AUC data points

DATA FILTERING - END


In [14]:
df_cmax = pk_regex.remove_low_quality_pk_data(df_cmax)

DATA FILTERING - START

Before curation, there are 24692 Cmax data points.
1143 Cmax data points have an ambiguous route of administration and were excluded.
1587 Cmax data points have no dose information but were kept.
119 Cmax data points have a dose range and they have been excluded.
After curation, there are now 23402 Cmax data points

DATA FILTERING - END


In [15]:
df_cmax = pk_regex.remove_low_quality_pk_data(df_cmax)

DATA FILTERING - START

Before curation, there are 23402 Cmax data points.
0 Cmax data points have an ambiguous route of administration and were excluded.
1587 Cmax data points have no dose information but were kept.
0 Cmax data points have a dose range and they have been excluded.
After curation, there are now 23402 Cmax data points

DATA FILTERING - END


---

#### Merge Cmax and AUC data 

In [16]:
df_pk = pd.concat([df_cmax, df_auc])

In [17]:
df_pk.shape

(44202, 9)

In [18]:
df_pk

,activity_id,standard_type,assay_id,description,assay_type,dose_unit,route,tissue,time
0,572270,Cmax,4422,Cmax 24 hr after 2 mg/kg oral administration i...,A,2 mg/kg,oral,,NaN
1,588108,Cmax,4422,Cmax 24 hr after 2 mg/kg oral administration i...,A,2 mg/kg,oral,,NaN
2,111183,Cmax,4423,Cmax in monkey after administration of 1 mg/kg iv,A,1 mg/kg,iv,,NaN
3,141171,Cmax,4423,Cmax in monkey after administration of 1 mg/kg iv,A,1 mg/kg,iv,,NaN
4,90201,Cmax,4424,Cmax was determine after peroral administratio...,A,10 mpk,peroral,,NaN
...,...,...,...,...,...,...,...,...,...
61770,24970535,AUC,2263024,"AUC in CD-1 mouse at 10 mg/kg,po measured upto...",A,10 mg/kg,po,,upto 1440 mins
61772,24970538,AUC,2263027,"AUC in CD-1 mouse at 5 mg/kg,iv measured upto ...",A,5 mg/kg,iv,,upto 1440 mins
61773,24970663,AUC,2263083,"AUC(0 to 24 hrs) in CD-1 mouse at 2 mg/kg,iv a...",A,2 mg/kg,iv,,AUC(0 to 24 hrs)
61775,24970667,AUC,2263087,"AUC(0 to 24 hrs) in CD-1 mouse at 10 mg/kg,po ...",A,10 mg/kg,po,,AUC(0 to 24 hrs)


#### Replace missing values by None

In [19]:
df_pk = df_pk.replace({'': None}).replace({np.nan: None})

#### Normalise the route of administration

We use the route of administration vocabulary than already used in ChEMBL

In [20]:
route_mapping = {'po':'ORAL','Po':'ORAL','PO':'ORAL','p.o.':'ORAL','Oral':'ORAL','orally':'ORAL'
                 ,'peroral':'ORAL','Peroral':'ORAL','perorally':'ORAL'
                 ,'paroral':'ORAL'
                 ,'Preoral':'ORAL'
                 ,'iv':'INTRAVENOUS','intravenously':'INTRAVENOUS','i.v.':'INTRAVENOUS','IV':'INTRAVENOUS'
                 ,'ip':'INTRAPERITONEAL','i.p.':'INTRAPERITONEAL','intraperitoneally':'INTRAPERITONEAL', 'IP':'INTRAPERITONEAL'
                 ,'sc':'SUBCUTANEOUS', 'SC': 'SUBCUTANEOUS'
                 ,'ig':'INTRAGASTRIC','IG':'INTRAGASTRIC','intragastrically':'INTRAGASTRIC'
                 ,'inhalation':'RESPIRATORY (INHALATION)'
                 ,'im':'INTRAMUSCULAR','IM':'INTRAMUSCULAR','intramuscularly':'INTRAMUSCULAR'
                 ,'intraduodenally':'INTRADUODENAL'
                 ,'intranasal':'NASAL', 'intranasally':'NASAL'
                 ,'intrajejunal':'INTRAILEAL'
                 ,'eye':'OPHTHALMIC','eyes':'OPHTHALMIC','intraocular':'INTRAOCULAR','intraocularly':'INTRAOCULAR'
                 ,'i.c.v':'INTRATHECAL'}

In [21]:
df_pk=df_pk.replace({"route": route_mapping})
df_pk['route'] = df_pk['route'].str.upper()

In [22]:
df_pk.route.unique()

array(['ORAL', 'INTRAVENOUS', None, 'INTRAPERITONEAL', 'SUBCUTANEOUS',
       'INTRAMUSCULAR', 'INTRADUODENAL', 'INTRAGASTRIC', 'OPHTHALMIC',
       'NASAL', 'SUBLINGUAL', 'NASOGASTRIC', 'INTRAILEAL',
       'RESPIRATORY (INHALATION)', 'INTRAOCULAR', 'INTRATHECAL'],
      dtype=object)

#### Normalise tissue 

In [23]:
df_pk = df_pk.assign(tissue=df_pk.tissue.str.lower().str.strip())

tissue_of_measurement = {
                        'csf':'cerebrospinal fluid'    
}

df_pk=df_pk.replace({"tissue": tissue_of_measurement})

df_pk.tissue.unique()

array([None, 'plasma', 'lung', 'blood', 'brain', 'kidney', 'liver',
       'spleen', 'cerebrospinal fluid', 'portal vein', 'serum', 'heart',
       'tumor', 'prostate', 'hypothalamus', 'biliary fluid', 'thigh',
       'retina', 'choroid', 'cornea', 'adipose tissue', 'skin',
       'interstitial fluid', 'muscle', 'cage fluid',
       'epithelial lining fluid', 'peritoneal fluid',
       'lung epithelial lining fluid', 'alveolar cell', 'body fluid',
       'thigh | interstitial fluid', 'urine', 'blood | portal vein',
       'mesenteric lymph', 'extracellular fluid', 'sciatic nerve',
       'small intestine', 'stomach', 'jejunum', 'thigh | muscle',
       'large intestine', 'fat', 'liver | tumor', 'tumor | kidney',
       'heart | tumor', 'liver | blood', 'intestine', 'tumor | serum'],
      dtype=object)

#### Format data 

In [24]:
df_pk = df_pk.melt(id_vars=['activity_id','standard_type','assay_id','description','assay_type'],
             value_vars=['route','time','dose_unit','tissue'])

In [25]:
df_pk.variable.unique()

array(['route', 'time', 'dose_unit', 'tissue'], dtype=object)

In [26]:
df_pk = df_pk.replace('dose_unit','DOSE')
df_pk = df_pk.replace('tissue','TISSUE')
df_pk = df_pk.replace('route','ROUTE')
df_pk = df_pk.replace('time','TIME')

In [27]:
df_pk.variable.unique()

array(['ROUTE', 'TIME', 'DOSE', 'TISSUE'], dtype=object)

In [28]:
df_pk = df_pk.rename({'value':'text_value', 'variable':'type'},axis=1)

In [29]:
df_pk = df_pk.assign(dose_unit=df_pk.text_value)

df_pk.loc[df_pk['type']!='DOSE','dose_unit'] = None
df_pk.loc[df_pk['type']=='DOSE','text_value'] = None

In [30]:
df_pk['value'], df_pk['units'] = zip(*df_pk.dose_unit.apply(lambda x: pk_regex.split_dose_unit(x)))
df_pk = df_pk.drop('dose_unit', axis=1)

In [31]:
df_pk.shape

(176808, 9)

In [32]:
df_pk = df_pk[~((df_pk.text_value.isnull())&(df_pk.value.isnull())&(df_pk.units.isnull()))].sort_values('activity_id')

In [33]:
df_pk.shape

(112452, 9)

#### AUC / Cmax stats 

In [34]:
df_pk[df_pk.standard_type=='Cmax']['activity_id'].nunique()

22792

In [35]:
df_pk[df_pk.standard_type=='Cmax'].groupby('type')['activity_id'].count().reset_index()

,type,activity_id
0,DOSE,21815
1,ROUTE,21260
2,TISSUE,5221


In [36]:
df_pk[df_pk.standard_type=='AUC']['activity_id'].nunique()

20800

In [37]:
df_pk[df_pk.standard_type=='AUC'].groupby('type')['activity_id'].count().reset_index()

,type,activity_id
0,DOSE,20029
1,ROUTE,19754
2,TIME,20800
3,TISSUE,3573


#### Normalise AUC time 

In [38]:
df_time = pd.read_csv('chembl32_regex_time_standardised_2023-05-05.csv')

In [39]:
df_time.relation.value_counts()

relation
<=    251
0      62
Name: count, dtype: int64

In [40]:
df_time.loc[~df_time.text_value.isnull(),'relation'] = None

In [41]:
df_time.relation.value_counts()

relation
<=    236
Name: count, dtype: int64

In [42]:
df_pk[(df_pk['type']=='TIME')].shape

(20800, 9)

In [43]:
df_pk = pd.concat(
            [df_pk[(df_pk['type']!='TIME')]
                , df_pk[(df_pk['type']=='TIME')].drop(
                       ['value','units']
                       , axis=1
                        ).merge(
                               df_time[['time','relation','value','units','text_value']]
                                , left_on = 'text_value'
                                , right_on='time'
                                , how='left').rename(
                                    {'text_value_y':'text_value'},axis=1).drop(
                                        ['text_value_x','time'],axis=1)
            ]
        )
df_pk = df_pk[~((df_pk.text_value.isnull())&(df_pk.value.isnull()))]
df_pk.shape

(112393, 10)

In [44]:
df_pk.loc[(df_pk['type']=='DOSE'),'relation'] = '='

In [45]:
df_pk[df_pk.standard_type=='Cmax']['activity_id'].nunique()

22792

In [46]:
df_pk.relation.value_counts()

relation
=     41844
<=     9951
Name: count, dtype: int64

In [47]:
def format_time_range(two_row_df):
    lower_time = two_row_df[two_row_df.value == two_row_df.value.min()]
    upper_time = two_row_df[two_row_df.value == two_row_df.value.max()]
    lower_time = lower_time.replace({'TIME':'TIME_LOWER'})
    upper_time = upper_time.replace({'TIME':'TIME_UPPER'})
    return(pd.concat([lower_time, upper_time]))
    

#### Rearrange time ranges so text_value does not contains any numerical vales (with the exception of 0-last) 

In [48]:
df_time_range = df_pk[(df_pk['type'] == 'TIME') & (~df_pk.text_value.isnull()) & (~df_pk.units.isnull())]

df_time_range['text_value'] = df_time_range.text_value.apply(lambda x: x.split(' - '))
df_time_range = df_time_range.explode('text_value')
df_time_range.loc[:, 'value'] = df_time_range.text_value
df_time_range.loc[:, 'text_value'] = None
df_time_range = df_time_range.groupby('activity_id').apply(lambda x : format_time_range(x)).droplevel(0)

df_pk = pd.concat([
                    df_pk[~((df_pk['type'] == 'TIME') & (~df_pk.text_value.isnull()) & (~df_pk.units.isnull()))]
                    , df_time_range
                    ])
df_pk.shape

(112775, 10)

In [49]:
df_pk = df_pk.replace({np.nan: None})

In [50]:
df_pk.text_value.unique()

array(['ORAL', None, 'blood', 'INTRAVENOUS', 'plasma', 'SUBCUTANEOUS',
       'cerebrospinal fluid', 'brain', 'INTRAPERITONEAL', 'INTRADUODENAL',
       'portal vein', 'tumor', 'INTRAMUSCULAR', 'heart', 'liver',
       'kidney', 'spleen', 'serum', 'lung', 'INTRAGASTRIC', 'prostate',
       'hypothalamus', 'biliary fluid', 'retina', 'OPHTHALMIC', 'choroid',
       'cornea', 'thigh', 'skin', 'NASAL', 'INTRAOCULAR',
       'epithelial lining fluid', 'cage fluid', 'muscle',
       'adipose tissue', 'interstitial fluid', 'SUBLINGUAL',
       'liver | blood', 'peritoneal fluid', 'alveolar cell',
       'lung epithelial lining fluid', 'body fluid', 'NASOGASTRIC',
       'thigh | interstitial fluid', 'urine', 'intestine',
       'mesenteric lymph', 'blood | portal vein', 'extracellular fluid',
       'heart | tumor', 'sciatic nerve', 'INTRAILEAL', 'small intestine',
       'stomach', 'RESPIRATORY (INHALATION)', 'INTRATHECAL', 'jejunum',
       'thigh | muscle', 'large intestine', 'fat', 'liver

In [51]:
df_pk.type.unique()

array(['ROUTE', 'DOSE', 'TISSUE', 'TIME', 'TIME_LOWER', 'TIME_UPPER'],
      dtype=object)

### Export flat file

In [52]:
df_pk.to_excel(f'chembl33_cmax_auc_legacy.xlsx', index=False)